# 0. set cuda disabe

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]=""

# 1. Import Library 

In [2]:
# Import TensorFlow >= 1.10 and enable eager execution
import tensorflow as tf
tf.enable_eager_execution()

import os
import time
import numpy as np
import matplotlib.pyplot as plt
import PIL
from IPython.display import clear_output

# 2. Load Image

In [3]:
path_to_zip = tf.keras.utils.get_file('facades.tar.gz',
                                      cache_subdir=os.path.abspath('.'),
                                      origin='https://people.eecs.berkeley.edu/~tinghuiz/projects/pix2pix/datasets/facades.tar.gz', 
                                      extract=True)

PATH = os.path.join(os.path.dirname(path_to_zip), 'facades/')

In [4]:
BUFFER_SIZE = 400
BATCH_SIZE = 1
IMG_WIDTH = 256
IMG_HEIGHT = 256

In [5]:
def load_image(image_file, is_train):
  image = tf.read_file(image_file)
  image = tf.image.decode_jpeg(image)

  w = tf.shape(image)[1]

  w = w // 2
  real_image = image[:, :w, :]
  input_image = image[:, w:, :]

  input_image = tf.cast(input_image, tf.float32)
  real_image = tf.cast(real_image, tf.float32)

  if is_train:
    # random jittering
    
    # resizing to 286 x 286 x 3
    input_image = tf.image.resize_images(input_image, [286, 286], 
                                        align_corners=True, 
                                        method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    real_image = tf.image.resize_images(real_image, [286, 286], 
                                        align_corners=True, 
                                        method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    
    # randomly cropping to 256 x 256 x 3
    stacked_image = tf.stack([input_image, real_image], axis=0)
    cropped_image = tf.random_crop(stacked_image, size=[2, IMG_HEIGHT, IMG_WIDTH, 3])
    input_image, real_image = cropped_image[0], cropped_image[1]

    if np.random.random() > 0.5:
      # random mirroring
      input_image = tf.image.flip_left_right(input_image)
      real_image = tf.image.flip_left_right(real_image)
  else:
    input_image = tf.image.resize_images(input_image, size=[IMG_HEIGHT, IMG_WIDTH], 
                                         align_corners=True, method=2)
    real_image = tf.image.resize_images(real_image, size=[IMG_HEIGHT, IMG_WIDTH], 
                                        align_corners=True, method=2)
  
  # normalizing the images to [-1, 1]
  input_image = (input_image / 127.5) - 1
  real_image = (real_image / 127.5) - 1

  return input_image, real_image

In [6]:
train_dataset = tf.data.Dataset.list_files(PATH+'train/*.jpg')
train_dataset = train_dataset.shuffle(BUFFER_SIZE)
train_dataset = train_dataset.map(lambda x: load_image(x, True))
train_dataset = train_dataset.batch(1)

In [7]:
test_dataset = tf.data.Dataset.list_files(PATH+'test/*.jpg')
test_dataset = test_dataset.map(lambda x: load_image(x, False))
test_dataset = test_dataset.batch(1)

In [8]:
test_dataset

<BatchDataset shapes: ((?, 256, 256, ?), (?, 256, 256, ?)), types: (tf.float32, tf.float32)>

# 3. Build Model

## Generator 

In [9]:
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, ReLU, LeakyReLU, BatchNormalization
from tensorflow.keras.models import Sequential

class Generator(tf.keras.Model):
    def __init__(self):
        super(Generator, self).__init__()

    #encoder
    def encoder(self, x):
        conv1 = Conv2D(64, [4,4] , padding='same', activation=ReLU())(x)
        conv2 = Conv2D(128, [4,4], padding='same', activation=ReLU())(conv1)
        bn = BatchNormalization()(conv2)
        return bn
    
    def decoder(self, x):
        deconv1 = Conv2DTranspose(128, [4,4], padding='same', activation=ReLU())(x)
        deconv2 = Conv2DTranspose( 64, [4,4], padding='same', activation=ReLU())(deconv1)
        return deconv2
    
    def call(self,x):
        print('call called')
        print(x.shape)
        return self.decoder(self.encoder(x))

In [10]:
x = tf.random_normal([4,256,256,3])
g = Generator()
fake_image = g(x)
print(fake_image.shape)


call called
(4, 256, 256, 3)
(4, 256, 256, 64)


## Discriminator 

In [11]:
class Discriminator(tf.keras.Model):
    def __init__(self):
        super(Discriminator, self).__init__()
        
    def discriminator(self, x):
        
        
    def __call__(self, x):
        
    

IndentationError: expected an indented block (<ipython-input-11-b03cb7cc1b68>, line 8)

## Loss 

## Compile

# 4. Training